In [20]:
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import col, current_timestamp

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 22, Finished, Available, Finished)

In [ ]:
# 1. Ler dados da pasta Silver
df_silver = spark.read.parquet("Files/silver/fi_ultimos2meses")

In [4]:
display(df_silver)

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9d9aca8d-e0ae-45a8-b73a-9cf57798ec41)

In [9]:
# 2. Adicionar data da carga
df_fact = df_silver.withColumn("DT_CARGA", current_timestamp())

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 11, Finished, Available, Finished)

In [28]:
# 3. Selecionar colunas da fato
fact_performance = df_fact.select(
    col("CNPJ_FUNDO_CLASSE").alias("CNPJ_FUNDO"),
    col("DT_COMPTC").alias("DT_COMPETENCIA"),
    col("VL_QUOTA").alias("VL_COTA"),
    col("VL_PATRIM_LIQ"),
    col("CAPTC_DIA").alias("CAPTACAO_DIA"),
    col("RESG_DIA").alias("RESGATE_DIA"),
    col("NR_COTST").alias("N_COTISTAS"),
    col("DT_CARGA")
)

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 30, Finished, Available, Finished)

In [29]:
display(fact_performance)

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9c14e9e0-47e3-4d02-93cf-db456c4904b7)

In [32]:
# 4. 📆 Calcular intervalo de datas
ontem = datetime.today() - timedelta(days=1)

# Início: dia 1 de 2 meses atrás
data_inicio = (ontem - relativedelta(months=2)).replace(day=1)

# Formatar datas para o replaceWhere
inicio_str = data_inicio.strftime("%Y-%m-%d")
fim_str = ontem.strftime("%Y-%m-%d")

print("Intervalo: ",inicio_str," <-> ", fim_str)

StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 34, Finished, Available, Finished)

Intervalo:  2025-05-01  <->  2025-07-18


In [33]:
# 5. Salvar como tabela no Lakehouse na camada Gold
fact_performance.write.mode("overwrite")\
    .option("replaceWhere", f"DT_COMPETENCIA BETWEEN '{inicio_str}' AND '{fim_str}'") \
    .saveAsTable("Fact_PerformanceDiaria")



StatementMeta(, 17b3f0e5-9b5a-42c6-9e4b-edf0e4b2425a, 35, Finished, Available, Finished)